In [49]:
import openai
import pandas as pd
from googlesearch import search
import wikipedia
from bs4 import BeautifulSoup
import re
#from gensim.summarization.summarizer import summarize
#from gensim.summarization import keywords
import urllib.request
import re
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## getting 'url's from google results

In [42]:
urls = []
#for i in search("putin", lang="en"):
#    urls.append(i)
for i in search(input(), lang="en"):
    urls.append(i)

putin


In [50]:
urls = list(set(urls))
urls

['https://en.wikipedia.org/wiki/Vladimir_Putin',
 'https://www.business-standard.com/about/who-is-vladimir-putin',
 'https://www.hindustantimes.com/world-news/vladimir-putin-news-moldova-russia-ukraine-war-news-vladimir-putin-planning-a-coup-in-this-country-us-says-deeply-concerned-101676355151895.html',
 'http://en.kremlin.ru/',
 'https://www.aljazeera.com/tag/vladimir-putin/',
 'https://www.britannica.com/biography/Vladimir-Putin',
 'https://www.wsj.com/topics/person/vladimir-putin',
 'https://www.cnn.com/2023/02/16/politics/ukraine-strategy-putin-west/index.html',
 'https://timesofindia.indiatimes.com/topic/Vladimir-Putin']

In [ ]:
# urls = ['https://en.wikipedia.org/wiki/Vladimir_Putin',
#  'https://www.britannica.com/biography/Vladimir-Putin',
#  'http://en.kremlin.ru/',
#  'https://www.hindustantimes.com/world-news/vladimir-putin-news-moldova-russia-ukraine-war-news-vladimir-putin-planning-a-coup-in-this-country-us-says-deeply-concerned-101676355151895.html',
#  'https://www.hindustantimes.com/world-news/vladimir-putin-news-moldova-russia-ukraine-war-news-vladimir-putin-planning-a-coup-in-this-country-us-says-deeply-concerned-101676355151895.html',
#  'https://www.hindustantimes.com/world-news/vladimir-putin-news-moldova-russia-ukraine-war-news-vladimir-putin-planning-a-coup-in-this-country-us-says-deeply-concerned-101676355151895.html',
#  'https://www.hindustantimes.com/world-news/vladimir-putin-news-moldova-russia-ukraine-war-news-vladimir-putin-planning-a-coup-in-this-country-us-says-deeply-concerned-101676355151895.html',
#  'https://www.reuters.com/world/europe/putin-secure-power-sets-stage-long-draining-war-2023-02-15/',
#  'https://www.reuters.com/world/europe/putin-secure-power-sets-stage-long-draining-war-2023-02-15/',
#  'https://www.reuters.com/world/europe/putin-secure-power-sets-stage-long-draining-war-2023-02-15/']

In [51]:
corpus = {}
for url in urls:
    request = urllib.request.Request(url)
    request.add_header('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
    raw_response = urllib.request.urlopen(request).read()
    soup = BeautifulSoup(raw_response)
    soup.find_all()
    text = ''
    new_text1 = ''
    new_text2 = ''
    new_text3 = ''
    for i in soup.find_all('p'):
        text +=str(i)
    text = text.replace('\n','')
    text = text.replace('\t','')
    p = re.compile(r'<[^>]+>')
    new_text1 = p.sub('', text).replace('\\','').replace('\\n','')
    #print(new_text)
    q = re.compile(r"\{.*\}")
    new_text2 = q.sub('',new_text1)
    r = re.compile(r'\[.*?\]')
    new_text3 = r.sub('',new_text2)
    corpus[url] = new_text3

## Summarization using openai api

In [52]:
# api key
openai.api_key = "your_api_key_generated_through_your_account"

In [53]:
summary = {}
for i in corpus.keys():
    prompt = corpus[i][0:3500].split('.')
    prompt = '.'.join(prompt[0:len(prompt)-1])
    prompt = prompt.replace('\n','')
    prompt = prompt+ ' Tl;dr'        # adding Tl;dr will start summarizing the code
    model_engine = "text-davinci-003"
    max_tokens = 800                # reduced the summary because the result is a summary and here in my case , each corpus is big
    completion = openai.Completion.create(
        engine=model_engine,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=0.5,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )
    summary[i] = completion.choices[0].text
    

## Printing the summary of all the links

In [74]:
print('\n'.join(summary.values()).replace('\\','').replace('\xad','').replace('Tl;dr stands for "too long; didn\'t read"',''))



Vladimir Putin is a Russian politician who has served as president or prime minister since 1999. He worked as a KGB foreign intelligence officer for 16 years before beginning his political career, and has been reelected in 2004, 2012, and 2018. Under his leadership, Russia has undergone democratic backsliding and a shift to authoritarianism, characterized by corruption and human rights violations. In April 2021, he signed into law constitutional amendments allowing him to potentially extend his presidency to 2036.


. It is an acronym used to indicate that a person has not read a long piece of text or is not interested in doing so.
 - US National Security Council spokesman John Kirby called reports alleging that Russian President Vladimir Putin is plotting a coup in Moldova "deeply concerning." The remarks came after Moldovan President Maia Sandu said that Putin is planning to use foreign saboteurs to destroy Moldova's leadership and stop it from joining the European Union. Russia re

## Sentiment analysis of each article on the given topic

In [54]:
sentiments = {}
for i in summary.keys():
    analyzer = SentimentIntensityAnalyzer()
    sentiments[i] = analyzer.polarity_scores(corpus[i])

In [55]:
sentiments

{'https://en.wikipedia.org/wiki/Vladimir_Putin': {'neg': 0.091,
  'neu': 0.815,
  'pos': 0.094,
  'compound': -0.7528},
 'https://www.business-standard.com/about/who-is-vladimir-putin': {'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0,
  'compound': 0.0},
 'https://www.hindustantimes.com/world-news/vladimir-putin-news-moldova-russia-ukraine-war-news-vladimir-putin-planning-a-coup-in-this-country-us-says-deeply-concerned-101676355151895.html': {'neg': 0.112,
  'neu': 0.85,
  'pos': 0.039,
  'compound': -0.9581},
 'http://en.kremlin.ru/': {'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0,
  'compound': 0.0},
 'https://www.aljazeera.com/tag/vladimir-putin/': {'neg': 0.073,
  'neu': 0.871,
  'pos': 0.056,
  'compound': -0.7543},
 'https://www.britannica.com/biography/Vladimir-Putin': {'neg': 0.039,
  'neu': 0.864,
  'pos': 0.097,
  'compound': 0.997},
 'https://www.wsj.com/topics/person/vladimir-putin': {'neg': 0.125,
  'neu': 0.82,
  'pos': 0.056,
  'compound': -0.9898},
 'https://www.cnn.com/2023/02/16/pol

## Sentiment DataFrame

In [59]:
sentiment_dataframe = pd.DataFrame(sentiments)
sentiment_dataframe.columns = [i.split('.')[1] for i in corpus.keys()]
sentiment_dataframe

,wikipedia,business-standard,hindustantimes,kremlin,aljazeera,britannica,wsj,cnn,indiatimes
neg,0.0910,0.0,0.1120,0.0,0.0730,0.039,0.1250,0.1470,0.0930
neu,0.8150,1.0,0.8500,1.0,0.8710,0.864,0.8200,0.7840,0.8200
pos,0.0940,0.0,0.0390,0.0,0.0560,0.097,0.0560,0.0690,0.0880
compound,-0.7528,0.0,-0.9581,0.0,-0.7543,0.997,-0.9898,-0.9988,-0.7703


# Majority of the websites have high negative compound polarity for the given input ( a person's name here i.e, putin)

In [76]:
','.join('''RUN_ID
FK_LOCATION_ID
LOCATION_CODE
FK_DATE_ID
RUN_DATE
FK_BOT_ID
BOT_ID
SCHEDULE_ID
SCHEDULE_TIMESTAMP
SCHEDULE_NAME
FK_STATUS_ID
RUN_STATUS
RUN_DURATION
NUM_TAGS_READ
AVG_CERTAINTY
REQUIRES_HANDSCAN
FK_ERROR_ID
FAILURE_REASON
SUCCESS_COUNT
FAILURE_COUNT
RUN_COMPLETION
RUN_ACCEPTED
RUN_QUALITY
EPC_COUNT_MET
EPC_COUNT
SUM_QUANTITY_RECORDED
SUM_QUANTITY_IPS
INVENTORY_ACCURACY
INSERTED_TS
MODIFIED_TS'''.split('\n'))

'RUN_ID,FK_LOCATION_ID,LOCATION_CODE,FK_DATE_ID,RUN_DATE,FK_BOT_ID,BOT_ID,SCHEDULE_ID,SCHEDULE_TIMESTAMP,SCHEDULE_NAME,FK_STATUS_ID,RUN_STATUS,RUN_DURATION,NUM_TAGS_READ,AVG_CERTAINTY,REQUIRES_HANDSCAN,FK_ERROR_ID,FAILURE_REASON,SUCCESS_COUNT,FAILURE_COUNT,RUN_COMPLETION,RUN_ACCEPTED,RUN_QUALITY,EPC_COUNT_MET,EPC_COUNT,SUM_QUANTITY_RECORDED,SUM_QUANTITY_IPS,INVENTORY_ACCURACY,INSERTED_TS,MODIFIED_TS'